In [1]:
# import required libriaries
from pathlib import Path
import json

from fpl_predictor.util import *

In [2]:
# path to data directory
#path = Path('/home/jupyter/fpl-prediction/data')
path = Path('./fpl_predictor/data')

In [3]:
# read in latest training set
df_train = pd.read_csv(path/'train_v8.csv', index_col=0, 
                       dtype={'season':str,
                              'comp':str,
                              'squad':str})

# path to current season
season_paths = [path/'2021-22']

# name for current season
season_names = ['2122']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('2122', 11, range(13,33))

In [4]:
# all_players#[all_players['full_name'] == 'Ben Davies']

In [5]:
df_train.tail(50)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
125145,Roberto Firmino,18,4,30,Liverpool,Tottenham Hotspur,1.958866,1.463462,False,1,...,0.0,0.0,0.0,8.0,11.0,72.7,11.0,0.0,1.0,1
125146,Robin Koch,18,2,90,Leeds,Arsenal,0.605178,1.278609,True,-1,...,0.0,0.0,0.0,34.0,46.0,73.9,28.0,0.0,0.0,1
125147,Rob Holding,18,2,0,Arsenal,Leeds,1.278609,0.605178,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
125148,Rodrigo Hernandez,18,3,67,Manchester City,Newcastle United,2.366110,0.502571,False,2,...,0.0,1.0,0.0,79.0,88.0,89.8,63.0,2.0,2.0,1
125149,Rodrigo Moreno,18,4,0,Leeds,Arsenal,0.605178,1.278609,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
125150,Romain Saïss,18,2,90,Wolverhampton Wanderers,Chelsea,0.831096,1.998680,True,7,...,0.0,1.0,0.0,36.0,46.0,78.3,40.0,0.0,1.0,1
125151,Romelu Lukaku,18,4,0,Chelsea,Wolverhampton Wanderers,1.998680,0.831096,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
125152,Romeo Lavia,18,3,0,Manchester City,Newcastle United,2.366110,0.502571,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
125153,Ross Barkley,18,3,0,Chelsea,Wolverhampton Wanderers,1.998680,0.831096,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
125154,Ruben Loftus-Cheek,18,3,0,Chelsea,Wolverhampton Wanderers,1.998680,0.831096,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [6]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == season_names[0]].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

19

In [7]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [8]:
df_latest.drop_duplicates(inplace=True)

In [9]:
df_latest.head(50)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season
0,Eric Bailly,19,2,0,Manchester United,Newcastle United,1.968535,0.502571,False,0,...,12580,1,1,0.0,-2,404,406,0,2021-12-27T20:00:00Z,2122
1,Keinan Davis,19,4,0,Aston Villa,Chelsea,0.942576,1.998680,True,0,...,231277,3,1,0.0,-17589,1040,18629,0,2021-12-26T17:30:00Z,2122
2,James Ward-Prowse,19,3,90,Southampton,West Ham United,0.563202,0.797993,False,13,...,163808,3,2,17.0,1991,5449,3458,0,2021-12-26T15:00:00Z,2122
3,Bruno Miguel Borges Fernandes,19,3,90,Manchester United,Newcastle United,1.968535,0.502571,False,1,...,1425591,1,1,2.0,57567,79314,21747,1,2021-12-27T20:00:00Z,2122
4,James McArthur,19,3,0,Crystal Palace,Tottenham Hotspur,0.545912,1.463462,False,0,...,132188,0,3,0.0,-10326,3051,13377,0,2021-12-26T15:00:00Z,2122
5,Paul Dummett,19,2,0,Newcastle United,Manchester United,0.502571,1.968535,True,0,...,956,1,1,0.0,4,95,91,0,2021-12-27T20:00:00Z,2122
6,Jairo Riedewald,19,3,0,Crystal Palace,Tottenham Hotspur,0.545912,1.463462,False,0,...,2563,0,3,0.0,-3,63,66,0,2021-12-26T15:00:00Z,2122
7,Alexis Mac Allister,19,3,84,Brighton and Hove Albion,Brentford,0.572189,0.481299,True,3,...,29170,0,2,11.0,-363,94,457,0,2021-12-26T20:00:00Z,2122
8,Alireza Jahanbakhsh,19,3,0,Brighton and Hove Albion,Brentford,0.572189,0.481299,True,0,...,103,0,2,0.0,-2,0,2,0,2021-12-26T20:00:00Z,2122
9,José Diogo Dalot Teixeira,19,2,90,Manchester United,Newcastle United,1.968535,0.502571,False,2,...,304281,1,1,2.0,34561,39575,5014,0,2021-12-27T20:00:00Z,2122


In [10]:
df_latest.shape

(450, 34)

In [11]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'team', 'play_proba']]

In [12]:
last_play_proba.head()

,player,team,play_proba
0,Jack Grealish,Manchester City,1.0
1,Kyle Walker,Manchester City,0.5
2,Ilkay Gündogan,Manchester City,1.0
3,Kevin De Bruyne,Manchester City,1.0
4,John Stones,Manchester City,1.0


In [13]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on=['player', 'team'], how='left')

In [14]:
df_latest.drop_duplicates(inplace=True)

In [15]:
df_train.drop_duplicates(inplace=True)

In [16]:
if last_gw==1:
    df_latest['relative_market_value_team_season'] = df_latest['relative_market_value_team']
    df_latest['relative_market_value_opponent_team_season'] = df_latest['relative_market_value_opponent_team']

In [17]:
# for when you're doing the season value properly
# relative_market_value_season = train_df[train_df['season'] == '2021'][['team', 'relative_market_value_team']].groupby(['team']).first()
# relative_market_value_season = relative_market_value_season.rename(columns={'relative_market_value_team': 'season_value'})
# relative_market_value_season

In [18]:
# now get futher stats from fbref
# standard stats
features = ['squad', 'comp', 'round', 'shots_total', 'shots_on_target', 'touches', 'pressures',
            'tackles', 'interceptions', 'blocks', 'xg', 'npxg', 'xa', 'sca', 
            'gca', 'passes_completed', 'passes', 'passes_pct',  'carries', 
            'dribbles_completed', 'dribbles']

In [19]:
# url for the premier league
url_base = 'https://fbref.com'
pl_history_page = '/en/comps/9/history/Premier-League-Seasons'
url = url_base + pl_history_page

# scrape the history table to get latest season
pl_history_table = get_table(url)
pl_history_urls = get_urls(['season'], 'season', pl_history_table, url_base)
season_urls = pl_history_urls[0:1]

df_enhanced = get_stats(features, season_urls, url_base)

2021-2022
https://fbref.com/en/comps/9/Premier-League-Stats
Manchester City
https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats
Ederson
João Cancelo
Bernardo Silva
Rúben Dias
Rodri
Aymeric Laporte
Gabriel Jesus
Kyle Walker
İlkay Gündoğan
Jack Grealish
Raheem Sterling
Phil Foden
Kevin De Bruyne
Riyad Mahrez
Oleksandr Zinchenko
Fernandinho
John Stones
Nathan Aké
Ferrán Torres
Cole Palmer
Zack Steffen
Benjamin Mendy
James Mcatee
Liverpool
https://fbref.com/en/squads/822bd0ba/Liverpool-Stats
Mohamed Salah
Sadio Mané
Alisson
Virgil van Dijk
Trent Alexander-Arnold
Joël Matip
Diogo Jota
Jordan Henderson
Andrew Robertson
Fabinho
Thiago Alcántara
Alex Oxlade-Chamberlain
Ibrahima Konaté
James Milner
Naby Keïta
Kostas Tsimikas
Roberto Firmino
Curtis Jones
Harvey Elliott
Caoimhín Kelleher
Tyler Morton
Takumi Minamino
Divock Origi
Joe Gomez
Neco Williams
Chelsea
https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats
Edouard Mendy
Antonio Rüdiger
Jorginho
Thiago Silva
Marcos Alonso
Reece James
Ma

In [20]:
# get the player name matching dictionary
with open(path/'match_list.json') as f:
  match_list = json.load(f)

match_list

{'Fabinho': 'Fabio Henrique Tavares',
 'Alisson': 'Alisson Ramses Becker',
 'Joël Matip': 'Joel Matip',
 'Joe Gomez': 'Joseph Gomez',
 'Naby Keïta': 'Naby Keita',
 'Adrián': 'Adrián San Miguel del Castillo',
 'Thiago Alcántara': 'Thiago Alcántara do Nascimento',
 'Allan': 'Allan Marques Loureiro',
 'Richarlison': 'Richarlison de Andrade',
 'Gylfi Sigurðsson': 'Gylfi Sigurdsson',
 'Séamus Coleman': 'Seamus Coleman',
 'André Gomes': 'André Filipe Tavares Gomes',
 'Bernard': 'Bernard Anício Caldeira Duarte',
 'Çağlar Söyüncü': 'Çaglar Söyüncü',
 'Bruno Fernandes': 'Bruno Miguel Borges Fernandes',
 'Fred': 'Frederico Rodrigues de Paula Santos',
 'Nemanja Matić': 'Nemanja Matic',
 'Alex Telles': 'Alex Nicolao Telles',
 'Pierre Højbjerg': 'Pierre-Emile Højbjerg',
 'Son Heung-min': 'Heung-Min Son',
 'Lucas Moura': 'Lucas Rodrigues Moura da Silva',
 'Érik Lamela': 'Erik Lamela',
 'Dele Alli': 'Bamidele Alli',
 'Carlos Vinícius': 'Carlos Vinicius Alves Morais',
 'Ederson': 'Ederson Santana de M

In [21]:
team_list = {}
team_list['Brighton'] = 'Brighton and Hove Albion'
team_list['Leeds United'] = 'Leeds'
team_list['Manchester Utd'] = 'Manchester United'
team_list['Newcastle Utd'] = 'Newcastle United'
team_list['Norwich City'] = 'Norwich'
team_list['Sheffield Utd'] = 'Sheffield United'
team_list['Tottenham'] = 'Tottenham Hotspur'
team_list['West Brom'] = 'West Bromwich Albion'
team_list['West Ham'] = 'West Ham United'
team_list['Wolves'] = 'Wolverhampton Wanderers'

In [22]:
# replace non-matching names/teams and extract gw as integer
df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)
df_enhanced['squad'] = df_enhanced['squad'].replace(team_list)
df_enhanced['round'] = df_enhanced['round'].str[-2:].astype('int')

In [23]:
# remove duplicated (happens when players transferred within season)
df_enhanced = df_enhanced.drop_duplicates()

In [24]:
df_enhanced

,season,player,date,squad,comp,round,shots_total,shots_on_target,touches,pressures,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,alt_name
0,2021-2022,Ederson,2021-08-15,Manchester City,Premier League,1,0,0,22,0,...,0.0,2,0,20,22,90.9,10,0,0,Ederson Santana de Moraes
1,2021-2022,Ederson,2021-08-21,Manchester City,Premier League,2,0,0,14,0,...,0.0,0,0,12,12,100.0,6,0,0,Ederson Santana de Moraes
2,2021-2022,Ederson,2021-08-28,Manchester City,Premier League,3,0,0,37,0,...,0.0,0,0,31,37,83.8,25,0,0,Ederson Santana de Moraes
3,2021-2022,Ederson,2021-09-11,Manchester City,Premier League,4,0,0,26,0,...,0.0,0,0,22,24,91.7,10,0,0,Ederson Santana de Moraes
4,2021-2022,Ederson,2021-09-18,Manchester City,Premier League,5,0,0,40,0,...,0.0,0,0,39,39,100.0,28,0,0,Ederson Santana de Moraes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4994,2021-2022,Angus Gunn,2021-12-28,Norwich,Premier League,20,0,0,31,0,...,0.0,0,0,21,27,77.8,16,0,0,Angus Gunn
4995,2021-2022,Sam Byram,2021-12-14,Norwich,Premier League,17,0,0,25,9,...,0.0,0,0,15,21,71.4,12,0,0,Sam Byram
4996,2021-2022,Sam Byram,2021-12-28,Norwich,Premier League,20,1,0,62,13,...,0.0,3,0,30,43,69.8,32,0,0,Sam Byram
4997,2021-2022,Bali Mumba,2021-08-21,Norwich,Premier League,2,0,0,19,12,...,0.0,0,0,7,10,70.0,9,0,1,Bali Mumba


In [25]:
# create date field in same format as enhanced data
df_latest['date'] = df_latest['kickoff_time'].str[:10]

In [26]:
# check if any names aren't matching
enhanced_players = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['alt_name'].unique().tolist()
standard_players = df_latest['player'].unique().tolist()

unmatched_players = [x for x in enhanced_players if x not in standard_players]
unmatched_players

[]

In [27]:
# check if any teams aren't matching
enhanced_teams = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['squad'].unique().tolist()
standard_teams = df_latest['team'].unique().tolist()

unmatched_teams = [x for x in enhanced_teams if x not in standard_teams]
unmatched_teams

[]

In [28]:
# any additional corrections
# match_list['Jakob Sørensen'] = 'Jacob Sørensen'
# match_list['Przemysław Płacheta'] = 'Przemyslaw Placheta'
# match_list['Mads Roerslev'] = 'Mads Roerslev Rasmussen'
# match_list['Odsonne Édouard'] = 'Odsonne Edouard'
# match_list['Hwang Hee-chan'] = 'Hee-Chan Hwang'

In [29]:
# replace additional non-matching names
# df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)

In [30]:
# make sure there are no non matches after update
# enhanced_players =  df_enhanced[df_enhanced['round'] == last_gw]['alt_name'].unique().tolist()
# standard_players = df_latest['player'].unique().tolist()

# unmatched_players = [x for x in enhanced_players if x not in standard_players]
# unmatched_players

In [31]:
# if match list was updated then update the file
# json_file = json.dumps(match_list)
# f = open(path/"match_list.json","w")
# f.write(json_file)
# f.close()

In [32]:
# check there are no duplicates in the join fields for the enhanced data
assert(len(df_enhanced[df_enhanced.duplicated(subset=['alt_name', 'date'], keep=False)]) == 0)

In [33]:
df_latest[df_latest.duplicated(subset=['date','player','team'], keep=False)]

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba,date


In [34]:
# check there are no duplicates in the join fields for the standard data
assert(len(df_train[df_train.duplicated(subset=['date','player','team'])]) == 0)

In [35]:
# df_train[df_train.duplicated(subset=['date','player','team'])]

In [36]:
# join standard and enhanced dataframes
df_latest_further = df_latest.merge(df_enhanced.drop(['player', 'season', 'round'], axis=1), 
                                   left_on=['date', 'player', 'team'], 
                                   right_on=['date', 'alt_name', 'squad'], how='left')

In [37]:
# drop the alt name field and check the length of the standard
# dataframe has not increased
df_latest_further = df_latest_further.drop(['alt_name'], axis=1)
assert(len(df_latest_further) == len(df_latest_further))

In [38]:
# add crowds field
df_latest_further['crowds'] = 1

In [39]:
assert(sum(df_latest_further[df_latest_further['minutes'] > 0]['xg'].isnull()) == 0)

In [40]:
df_latest_further[df_latest_further['minutes'] > 0][df_latest_further[df_latest_further['minutes'] > 0]['xg'].isnull()]

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds


In [41]:
df_train_new = pd.concat([df_train, df_latest_further], ignore_index=True, axis=0, sort=False)

In [42]:
df_train_new[df_train_new['player'] == 'Harry Kane'].tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
121926,Harry Kane,12,4,90,Tottenham Hotspur,Leeds,1.535073,0.569761,True,2,...,0.0,2.0,1.0,20.0,24.0,83.3,32.0,3.0,6.0,1
122835,Harry Kane,14,4,90,Tottenham Hotspur,Brentford,1.535582,0.440358,True,2,...,0.5,6.0,1.0,22.0,31.0,71.0,31.0,2.0,3.0,1
123471,Harry Kane,15,4,90,Tottenham Hotspur,Norwich,1.535539,0.410615,True,2,...,0.1,2.0,1.0,13.0,21.0,61.9,22.0,2.0,4.0,1
125020,Harry Kane,18,4,90,Tottenham Hotspur,Liverpool,1.463462,1.958866,True,6,...,0.2,2.0,0.0,10.0,18.0,55.6,20.0,3.0,3.0,1
125561,Harry Kane,19,4,63,Tottenham Hotspur,Crystal Palace,1.463462,0.545912,True,6,...,0.0,1,1,17,19,89.5,23,1,1,1


In [43]:
df_train_new.tail(10)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
125635,Scott Carson,19,1,0,Manchester City,Leicester City,2.366110,1.173500,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
125636,Federico Fernández,19,2,0,Newcastle United,Manchester United,0.502571,1.968535,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
125637,Mohammed Salisu,19,2,90,Southampton,West Ham United,0.563202,0.797993,False,1,...,0.0,0,0,24,33,72.7,19,0,0,1
125638,N'Golo Kanté,19,3,62,Chelsea,Aston Villa,1.998680,0.942576,False,2,...,0.0,3,1,48,52,92.3,40,2,3,1
125639,Josh Martin,19,3,0,Norwich,Arsenal,0.401897,1.278609,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
125640,Wilfred Ndidi,19,3,0,Leicester City,Manchester City,1.173500,2.366110,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
125641,Matt Ritchie,19,2,0,Newcastle United,Manchester United,0.502571,1.968535,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
125642,Nathan Redmond,19,3,90,Southampton,West Ham United,0.563202,0.797993,False,2,...,0.2,2,0,35,43,81.4,39,0,0,1
125643,Mathew Ryan,19,1,0,Brighton and Hove Albion,Brentford,0.572189,0.481299,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
125644,Ryan Fredericks,19,2,0,West Ham United,Southampton,0.797993,0.563202,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [44]:
df_latest_further.shape

(450, 57)

In [45]:
df_train_new.shape

(125645, 59)

In [46]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train_v8.csv')

In [47]:
# get fixtures
fixtures = pd.read_csv(path/'2021-22/fixtures.csv')
team_dict = teams[['team_2122', 'team']].dropna().set_index('team_2122').to_dict()['team']

fixtures['home_team'] = fixtures['team_h'].replace(team_dict)
fixtures['away_team'] = fixtures['team_a'].replace(team_dict)
fixtures = fixtures[['home_team', 'away_team', 'kickoff_time', 'event']]
fixtures = fixtures.rename(columns={'event':'gw'})


# use these two lines if updating play_proba later in week
# current_gw = last_gw
# df_train_new = pd.read_csv(path/'train_v7.csv', index_col=0, dtype={'season':str,
#                                                                     'comp':str,
#                                                                     'squad':str})

# set starting gameweek (where are we right now in the season)
current_gw = last_gw + 1
fixtures = fixtures[fixtures['gw'] >= current_gw].sort_values(by=['gw'])
fixtures['match_no'] = np.arange(fixtures.shape[0])

In [48]:
fixtures.head(50)

,home_team,away_team,kickoff_time,gw,match_no
195,Crystal Palace,Norwich,2021-12-28T15:00:00Z,20.0,0
196,Southampton,Tottenham Hotspur,2021-12-28T15:00:00Z,20.0,1
197,Watford,West Ham United,2021-12-28T15:00:00Z,20.0,2
198,Leicester City,Liverpool,2021-12-28T20:00:00Z,20.0,3
199,Chelsea,Brighton and Hove Albion,2021-12-29T19:30:00Z,20.0,4
200,Brentford,Manchester City,2021-12-29T20:15:00Z,20.0,5
201,Everton,Newcastle United,2021-12-30T19:30:00Z,20.0,6
202,Manchester United,Burnley,2021-12-30T20:15:00Z,20.0,7
212,Manchester United,Wolverhampton Wanderers,2022-01-03T17:30:00Z,21.0,8
211,Chelsea,Liverpool,2022-01-02T16:30:00Z,21.0,9


In [49]:
# add universal team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'match_no', 'home_team', 'away_team', 'kickoff_time', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [50]:
fixtures.head()

,gw,match_no,home_team,away_team,kickoff_time,home_team_code,away_team_code
0,20.0,0,Crystal Palace,Norwich,2021-12-28T15:00:00Z,31,45
1,20.0,1,Southampton,Tottenham Hotspur,2021-12-28T15:00:00Z,20,6
2,20.0,2,Watford,West Ham United,2021-12-28T15:00:00Z,57,21
3,20.0,3,Leicester City,Liverpool,2021-12-28T20:00:00Z,13,14
4,20.0,4,Chelsea,Brighton and Hove Albion,2021-12-29T19:30:00Z,8,36


In [51]:
# all_players.replace(to_replace='None', value=1, inplace=True)

In [52]:
all_players[all_players['play_proba_2122'] == '0']

,full_name,play_proba_2122,position_2122,id_2122,cost_2122,team_2122,web_name_2122
1,Rúnar Alex Rúnarsson,0,1,2,40,3,Rúnarsson
2,Willian Borges Da Silva,0,3,3,63,3,Willian
8,Héctor Bellerín,0,2,9,48,3,Bellerín
10,Sead Kolasinac,0,2,11,43,3,Kolasinac
17,Lucas Torreira,0,3,18,45,3,Torreira
...,...,...,...,...,...,...,...
649,Andreas Söndergaard,0,1,444,39,39,Söndergaard
651,Yerson Mosquera Valdelamar,0,2,446,44,39,Mosquera
656,Patrick Cutrone,0,4,523,50,39,Cutrone
659,Christian Marques,0,2,549,40,39,Marques


In [53]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_2122', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'match_no', 'home_team', 'away_team', 'kickoff_time', 
                   'full_name', 'position_2122', 'cost_2122', 'play_proba_2122', 
                   'web_name_2122']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_2122':'position',
                        'cost_2122':'price',
                        'play_proba_2122':'play_proba',
                        'web_name_2122':'short_name'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [54]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_2122', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'match_no', 'away_team', 'home_team', 'kickoff_time', 
                   'full_name', 'position_2122', 'cost_2122', 'play_proba_2122', 
                   'web_name_2122']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_2122':'position',
                        'cost_2122':'price',
                        'play_proba_2122':'play_proba',
                        'web_name_2122':'short_name'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [55]:
# look at away players
away_df.head()

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home
0,20.0,0,Norwich,Crystal Palace,2021-12-28T15:00:00Z,Billy Gilmour,3,44,100,Gilmour,False
1,20.0,0,Norwich,Crystal Palace,2021-12-28T15:00:00Z,Brandon Williams,2,39,100,Williams,False
2,20.0,0,Norwich,Crystal Palace,2021-12-28T15:00:00Z,Michael McGovern,1,40,None,McGovern,False
3,20.0,0,Norwich,Crystal Palace,2021-12-28T15:00:00Z,Tim Krul,1,45,25,Krul,False
4,20.0,0,Norwich,Crystal Palace,2021-12-28T15:00:00Z,Teemu Pukki,4,59,None,Pukki,False


In [56]:
# look at home players
home_df.head()

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home
0,20.0,0,Crystal Palace,Norwich,2021-12-28T15:00:00Z,Conor Gallagher,3,61,100,Gallagher,True
1,20.0,0,Crystal Palace,Norwich,2021-12-28T15:00:00Z,Vicente Guaita,1,46,None,Guaita,True
2,20.0,0,Crystal Palace,Norwich,2021-12-28T15:00:00Z,James Tomkins,2,43,None,Tomkins,True
3,20.0,0,Crystal Palace,Norwich,2021-12-28T15:00:00Z,James McArthur,3,43,0,McArthur,True
4,20.0,0,Crystal Palace,Norwich,2021-12-28T15:00:00Z,Christian Benteke,4,63,None,Benteke,True


In [57]:
# # calculate average minutes over last 5 gameweeks
# # first get last 5 gameweeks in current season
# recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
#                            (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# # recent_mins = df_train_new[((df_train_new['gw'] >= 40 - 5) & 
# #                            (df_train_new['season'] == '1920')) | 
# #                            ((df_train_new['gw'] >= 1) & 
# #                            (df_train_new['season'] == '2021'))][['player', 'minutes', 'play_proba']]

# # exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
# recent_mins = recent_mins[recent_mins['play_proba'] == 1]

# len(recent_mins)

# # average for each player
# recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

# recent_mins

In [58]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2122'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2122'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = season_names[0]

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# # add minutes based on last five fixtures
# remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
# remaining_season_df['minutes'].fillna(0, inplace=True)

# # multiply minutes by play probability
# remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']

In [59]:
remaining_season_df.head(50)

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home,relative_market_value_team,relative_market_value_opponent_team,season
0,20.0,0,Crystal Palace,Norwich,2021-12-28T15:00:00Z,Conor Gallagher,3,6.1,1.00,Gallagher,True,0.545912,0.401897,2122
1,20.0,0,Crystal Palace,Norwich,2021-12-28T15:00:00Z,Vicente Guaita,1,4.6,1.00,Guaita,True,0.545912,0.401897,2122
2,20.0,0,Crystal Palace,Norwich,2021-12-28T15:00:00Z,James Tomkins,2,4.3,1.00,Tomkins,True,0.545912,0.401897,2122
3,20.0,0,Crystal Palace,Norwich,2021-12-28T15:00:00Z,James McArthur,3,4.3,0.00,McArthur,True,0.545912,0.401897,2122
4,20.0,0,Crystal Palace,Norwich,2021-12-28T15:00:00Z,Christian Benteke,4,6.3,1.00,Benteke,True,0.545912,0.401897,2122
5,20.0,0,Crystal Palace,Norwich,2021-12-28T15:00:00Z,Cheikhou Kouyaté,2,4.5,1.00,Kouyaté,True,0.545912,0.401897,2122
6,20.0,0,Crystal Palace,Norwich,2021-12-28T15:00:00Z,Martin Kelly,2,3.9,1.00,Kelly,True,0.545912,0.401897,2122
7,20.0,0,Crystal Palace,Norwich,2021-12-28T15:00:00Z,Luka Milivojevic,3,5.3,1.00,Milivojevic,True,0.545912,0.401897,2122
8,20.0,0,Crystal Palace,Norwich,2021-12-28T15:00:00Z,Jordan Ayew,4,5.8,1.00,Ayew,True,0.545912,0.401897,2122
9,20.0,0,Crystal Palace,Norwich,2021-12-28T15:00:00Z,Wilfried Zaha,3,6.8,0.00,Zaha,True,0.545912,0.401897,2122


In [60]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')